In [ ]:
import pandas as pd

try:
    df = pd.read_csv('yandex_reviews_clean.csv')
    print("Файл успешно загружен!")

    display(df.head())
    
    print("\nИнфо о данных:")
    df.info()

except FileNotFoundError:
    print("Ошибка: Файл не найден. Проверьте, загрузили ли вы его в меню слева.")

Ошибка: Файл не найден. Проверьте, загрузили ли вы его в меню слева.


In [3]:
!pip install pymorphy2

  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.5/55.5 kB 4.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.2/8.2 MB 65.6 MB/s eta 0:00:00
  Created wheel for docopt: filename=docopt-0.6.2-py2.py3-none-any.whl size=13706 sha256=f197ab70f6038ad2ac4e48d4282401d8db1774591b08dabeb2f785dc0941a62a
  Stored in directory: /root/.cache/pip/wheels/1a/bf/a1/4cee4f7678c68c5875ca89eaccf460593539805c3906722228
Successfully built docopt


In [ ]:
import pandas as pd
from collections import Counter

data = {
    'date': ['2025-12-09'],
    'rating': [1],
    'text': ['Дорого и грубый персонал']
}
df_example = pd.DataFrame(data)

df_example['lemmas'] = None

print("1. Структура Таблицы (пока пустая колонка lemmas):")
display(df_example)

processed_words = ['дорого', 'грубый', 'персонал']

df_example.at[0, 'lemmas'] = processed_words

print("\n2. Таблица заполнена списком слов:")
display(df_example)

all_words = ['дорого', 'грубый', 'персонал', 'дорого', 'очередь']

word_counts = Counter(all_words)

print("\n3. Структура Частотного словаря (Слово -> Количество):")
print(word_counts)

print(f"Слово 'дорого' встречается: {word_counts['дорого']} раз")

1. Структура Таблицы (пока пустая колонка lemmas):


,date,rating,text,lemmas
0,2025-12-09,1,Дорого и грубый персонал,None



2. Таблица заполнена списком слов:


,date,rating,text,lemmas
0,2025-12-09,1,Дорого и грубый персонал,"[дорого, грубый, персонал]"



3. Структура Частотного словаря (Слово -> Количество):
Counter({'дорого': 2, 'грубый': 1, 'персонал': 1, 'очередь': 1})
Слово 'дорого' встречается: 2 раз


In [ ]:
import pandas as pd

try:
    df = pd.read_csv('yandex_reviews_clean.csv')
    print(f"1. Файл загружен. Всего строк: {len(df)}")
except FileNotFoundError:
    print("Ошибка: Файл не найден. Убедитесь, что вы загрузили его в Colab.")

df = df.dropna(subset=['rating']).copy()
df = df.dropna(subset=['text']).copy()
df = df.reset_index(drop=True)

print(f"2. После удаления мусора осталось строк: {len(df)}")

months_map = {
    'января': '01', 'февраля': '02', 'марта': '03', 'апреля': '04',
    'мая': '05', 'июня': '06', 'июля': '07', 'августа': '08',
    'сентября': '09', 'октября': '10', 'ноября': '11', 'декабря': '12'
}

def parse_russian_date(date_str):
    """Функция превращает '9 декабря 2025' в дату, понятную компьютеру"""
    if pd.isna(date_str):
        return None
    try:
        parts = date_str.split()
        if len(parts) >= 3:
            day = parts[0].zfill(2)
            month = months_map.get(parts[1].lower())
            year = parts[2]
            if month:
                return pd.to_datetime(f"{year}-{month}-{day}")
    except:
        return None
    return None

def get_season(date_obj):
    """Функция определяет сезон по месяцу"""
    if pd.isna(date_obj):
        return None
    m = date_obj.month
    if m in [12, 1, 2]: return 'Зима'
    elif m in [3, 4, 5]: return 'Весна'
    elif m in [6, 7, 8]: return 'Лето'
    else: return 'Осень'

df['date_clean'] = df['date'].apply(parse_russian_date)
df['season'] = df['date_clean'].apply(get_season)

print("\nПоказываю пример данных (первые 5 строк):")
display(df[['date', 'date_clean', 'season', 'rating', 'text']].head())

Ошибка: Файл не найден. Убедитесь, что вы загрузили его в Colab.


NameError: name 'df' is not defined

In [ ]:
from google.colab import data_table
from google.colab import files

data_table.enable_dataframe_formatter()

print("Ниже интерактивная таблица. Можно листать страницы и кликать по заголовкам для сортировки:")
display(df)
output_filename = 'reviews_step1_dates.csv'
df.to_csv(output_filename, index=False)

print(f"\nФайл {output_filename} сформирован.")
print("Попытка автоматического скачивания...")

try:
    files.download(output_filename)
except Exception as e:
    print("Не удалось скачать автоматически (иногда браузер блокирует).")
    print("Вы можете скачать файл вручную: нажмите на значок Папки слева -> наведите на файл -> три точки -> Скачать.")

In [ ]:
import re 

df['text_lower'] = df['text'].astype(str).str.lower()

print("3. Столбец 'text_lower' создан.")

def tokenize(text):
    """
    1. Заменяет всё, что НЕ русская буква (знаки, смайлы, цифры), на пробел.
    2. Разбивает текст по пробелам на список слов.
    """
    if not isinstance(text, str):
        return []
    text_clean = re.sub(r'[^а-яё\s]', ' ', text)
    return text_clean.split()

df['tokens'] = df['text_lower'].apply(tokenize)

print("4. Столбец 'tokens' создан (текст разбит на слова).")

display(df[['text', 'text_lower', 'tokens']].head())

In [ ]:
from google.colab import data_table
from google.colab import files

data_table.enable_dataframe_formatter()

print("Ниже интерактивная таблица. Можно листать страницы и кликать по заголовкам для сортировки:")
display(df)
output_filename = 'reviews_step2_dates.csv'
df.to_csv(output_filename, index=False)

print(f"\nФайл {output_filename} сформирован.")
print("Попытка автоматического скачивания...")

try:
    files.download(output_filename)
except Exception as e:
    print("Не удалось скачать автоматически (иногда браузер блокирует).")
    print("Вы можете скачать файл вручную: нажмите на значок Папки слева -> наведите на файл -> три точки -> Скачать.")

In [ ]:
!pip install pymorphy2

In [ ]:
import pymorphy2
import inspect
def _getargspec_dummy(func):
    spec = inspect.getfullargspec(func)
    return (spec.args, spec.varargs, spec.varkw, spec.defaults)

inspect.getargspec = _getargspec_dummy


morph = pymorphy2.MorphAnalyzer()

stop_words = {
    'и', 'в', 'во', 'не', 'что', 'он', 'на', 'я', 'с', 'со', 'как', 'а', 'то', 'все', 'она', 'так', 'его', 'но', 'да', 'ты', 'к', 'у', 'же', 'вы', 'за', 'бы', 'по', 'только', 'ее', 'мне', 'было', 'вот', 'от', 'меня', 'еще', 'нет', 'о', 'из', 'ему', 'теперь', 'когда', 'даже', 'ну', 'вдруг', 'ли', 'если', 'уже', 'или', 'ни', 'быть', 'был', 'него', 'до', 'вас', 'нибудь', 'опять', 'уж', 'вам', 'ведь', 'там', 'потом', 'себя', 'ей', 'может', 'они', 'тут', 'где', 'есть', 'надо', 'ней', 'для', 'мы', 'тебя', 'их', 'чем', 'была', 'сам', 'чтоб', 'без', 'будто', 'чего', 'раз', 'тоже', 'себе', 'под', 'будет', 'ж', 'тогда', 'кто', 'этот', 'того', 'потому', 'этого', 'какой', 'совсем', 'ним', 'здесь', 'этом', 'один', 'почти', 'мой', 'тем', 'чтобы', 'нее', 'сейчас', 'были', 'куда', 'зачем', 'всех', 'можно', 'при', 'наконец', 'два', 'об', 'другой', 'хоть', 'после', 'над', 'тот', 'через', 'эти', 'нас', 'про', 'всего', 'них', 'какая', 'много', 'разве', 'три', 'эту', 'моя', 'впрочем', 'этой', 'перед', 'иногда', 'чуть', 'том', 'нельзя', 'такой', 'им', 'более',
    'аптека', 'аптеки'
}

def get_lemmas(tokens_list):
    """Превращает ['цены', 'высокие'] в ['цена', 'высокий']"""
    if not isinstance(tokens_list, list):
        return []

    norm_list = []
    for word in tokens_list:
        norm_list.append(morph.parse(word)[0].normal_form)
    return norm_list

print("5. Запущена лемматизация (приведение к начальной форме)... Ждите.")
df['lemmas_all'] = df['tokens'].apply(get_lemmas)
print("Готово! Столбец 'lemmas_all' создан.")

def filter_stops(lemmas_list):
    """
    Удаляет слова из списка stop_words и короткие слова (<3 букв).
    Превращает ['цена', 'высокий', 'и', 'в'] в ['цена', 'высокий']
    """
    clean_list = []
    for word in lemmas_list:
        if word not in stop_words and len(word) > 2:
            clean_list.append(word)
    return clean_list

print("6. Удаление стоп-слов...")
df['lemmas'] = df['lemmas_all'].apply(filter_stops)
print("Готово! Столбец 'lemmas' (финальный) создан.")

display(df[['tokens', 'lemmas_all', 'lemmas']].head())

In [ ]:
from google.colab import files

df.to_csv('reviews_processed_final.csv', index=False)
print("Файл reviews_processed_final.csv создан.")
files.download('reviews_processed_final.csv')

In [ ]:
import pandas as pd
import ast
from collections import Counter

filename = 'reviews_processed_final.csv'

try:
    print(f"Попытка загрузить {filename}...")
    df = pd.read_csv(filename)

    print("Восстанавливаю списки слов (это может занять пару секунд)...")
    df['lemmas'] = df['lemmas'].apply(ast.literal_eval)

    print(f"Файл загружен! Строк: {len(df)}")

except FileNotFoundError:
    print(f"ОШИБКА: Файл {filename} не найден. Загрузите его в папку слева или выполните предыдущие шаги.")


def collect_words(dataframe):
    """Собирает все слова из колонки lemmas в один большой список"""
    all_words_list = []
    for row_list in dataframe['lemmas']:
        all_words_list.extend(row_list)
    return all_words_list

if 'df' in locals():
    all_lemmas = collect_words(df)
    counter_all = Counter(all_lemmas)
    print(f"\n1. Общая база готова. Топ-10 слов: {counter_all.most_common(10)}")

    df_pos = df[df['rating'] >= 4]
    pos_lemmas = collect_words(df_pos)
    counter_pos = Counter(pos_lemmas)
    print(f"2. Позитивная база готова. Топ-10 слов: {counter_pos.most_common(10)}")

    df_neg = df[df['rating'] <= 3]
    neg_lemmas = collect_words(df_neg)
    counter_neg = Counter(neg_lemmas)
    print(f"3. Негативная база готова. Топ-10 слов: {counter_neg.most_common(10)}")

    topics_dict = {
        'Сервис': ['персонал', 'фармацевт', 'сотрудник', 'вежливый', 'обслуживание', 'хамство', 'грубый', 'приветливый', 'помочь'],
        'Очереди/Скорость': ['очередь', 'быстро', 'медленно', 'ждать', 'время', 'долго', 'окно', 'касса', 'работать'],
        'Ассортимент': ['наличие', 'ассортимент', 'лекарство', 'препарат', 'купить', 'найти', 'заказ', 'аналог'],
        'Цена': ['цена', 'дорого', 'дешево', 'скидка', 'акция', 'бонус', 'стоимость', 'ценник', 'деньги'],
        'Помещение': ['чисто', 'грязно', 'зал', 'вход', 'пандус', 'уютно']
    }
    print("\n4. Словарь категорий (topics_dict) создан.")

else:
    print("Не удалось продолжить из-за ошибки загрузки файла.")

Попытка загрузить reviews_processed_final.csv...
Восстанавливаю списки слов (это может занять пару секунд)...
Файл загружен! Строк: 394

1. Общая база готова. Топ-10 слов: [('заказ', 313), ('лекарство', 133), ('доставка', 118), ('это', 103), ('цена', 102), ('всё', 91), ('день', 88), ('хороший', 80), ('очень', 78), ('курьер', 71)]
2. Позитивная база готова. Топ-10 слов: [('цена', 70), ('хороший', 58), ('очень', 57), ('лекарство', 49), ('заказ', 44), ('всё', 40), ('всегда', 36), ('вежливый', 28), ('персонал', 27), ('наличие', 25)]
3. Негативная база готова. Топ-10 слов: [('заказ', 269), ('доставка', 113), ('это', 94), ('лекарство', 84), ('день', 84), ('курьер', 70), ('сделать', 54), ('всё', 51), ('мочь', 50), ('заказать', 48)]

4. Словарь категорий (topics_dict) создан.


In [ ]:
extra_stop_words = {'это', 'всё', 'мочь', 'очень'}
def remove_extra_stops(lemmas_list):
    return [word for word in lemmas_list if word not in extra_stop_words]

df['lemmas'] = df['lemmas'].apply(remove_extra_stops)

print("Лишние слова удалены из таблицы.")


def collect_words(dataframe):
    all_words_list = []
    for row_list in dataframe['lemmas']:
        all_words_list.extend(row_list)
    return all_words_list

all_lemmas = collect_words(df)
counter_all = Counter(all_lemmas)

df_pos = df[df['rating'] >= 4]
pos_lemmas = collect_words(df_pos)
counter_pos = Counter(pos_lemmas)

df_neg = df[df['rating'] <= 3]
neg_lemmas = collect_words(df_neg)
counter_neg = Counter(neg_lemmas)

print("\n--- НОВЫЕ ТОПЫ (ЧИСТЫЕ) ---")
print(f"Общее: {counter_all.most_common(10)}")
print(f"Позитив: {counter_pos.most_common(10)}")
print(f"Негатив: {counter_neg.most_common(10)}")

topics_dict = {
    'Доставка/Заказ': [
        'заказ', 'доставка', 'курьер', 'привезти', 'опоздать', 'ждать', 'время',
        'отменить', 'срок', 'приложение', 'сайт', 'интернет'
    ],
    'Лекарства/Наличие': [
        'лекарство', 'препарат', 'наличие', 'таблетки', 'аналог', 'срок', 'годность',
        'упаковка', 'ампула'
    ],
    'Цена/Выгода': [
        'цена', 'дорого', 'дешево', 'стоимость', 'деньги', 'скидка', 'акция',
        'бонус', 'промокод', 'рубль', 'вернуть'
    ],
    'Сервис/Общение': [
        'вежливый', 'хамство', 'персонал', 'фармацевт', 'трубка', 'дозвониться',
        'оператор', 'поддержка', 'помочь', 'спасибо'
    ]
}
print("\nСловарь тем обновлен под специфику интернет-заказов.")

Лишние слова удалены из таблицы.

--- НОВЫЕ ТОПЫ (ЧИСТЫЕ) ---
Позитив: [('заказ', 313), ('лекарство', 133), ('доставка', 118), ('цена', 102), ('день', 88), ('хороший', 80), ('курьер', 71), ('заказать', 66), ('препарат', 58), ('сделать', 57)]
Позитив: [('цена', 70), ('хороший', 58), ('лекарство', 49), ('заказ', 44), ('всегда', 36), ('вежливый', 28), ('персонал', 27), ('наличие', 25), ('быстро', 25), ('очередь', 23)]
Негатив: [('заказ', 269), ('доставка', 113), ('лекарство', 84), ('день', 84), ('курьер', 70), ('сделать', 54), ('заказать', 48), ('препарат', 42), ('время', 42), ('который', 41)]

Словарь тем обновлен под специфику интернет-заказов.


In [ ]:
import pandas as pd
import json
from google.colab import files

df.to_csv('1_reviews_full_cleaned.csv', index=False)
print("Файл 1_reviews_full_cleaned.csv создан (Главная таблица).")

def save_counter_to_csv(counter, filename):
    temp_df = pd.DataFrame(counter.most_common(), columns=['word', 'count'])
    temp_df.to_csv(filename, index=False)
    print(f"Файл {filename} создан.")

save_counter_to_csv(counter_all, '2_stats_general.csv') 
save_counter_to_csv(counter_pos, '3_stats_positive.csv')
save_counter_to_csv(counter_neg, '4_stats_negative.csv')

with open('5_topics_config.json', 'w', encoding='utf-8') as f:
    json.dump(topics_dict, f, ensure_ascii=False, indent=4)
print("Файл 5_topics_config.json создан (Словарь тем).")

files.download('1_reviews_full_cleaned.csv')
files.download('2_stats_general.csv')
files.download('3_stats_positive.csv')
files.download('4_stats_negative.csv')
files.download('5_topics_config.json')

Файл 1_reviews_full_cleaned.csv создан (Главная таблица).
Файл 2_stats_general.csv создан.
Файл 3_stats_positive.csv создан.
Файл 4_stats_negative.csv создан.
Файл 5_topics_config.json создан (Словарь тем).


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

...

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
import pandas as pd
import ast

df = pd.read_csv('1_reviews_full_cleaned.csv')
df['lemmas'] = df['lemmas'].apply(ast.literal_eval)

def get_topic(word):
    for topic, words_list in topics_dict.items():
        if word in words_list:
            return topic
    return 'Другое'

print("Подготовка файла для DataLens...")

new_rows = []

for index, row in df.iterrows():
    sentiment = 'Позитив' if row['rating'] >= 4 else ('Негатив' if row['rating'] <= 3 else 'Нейтрально')

    for word in row['lemmas']:
        new_rows.append({
            'date_clean': row['date_clean'],
            'season': row['season'],        
            'rating': row['rating'],        
            'sentiment': sentiment,         
            'word': word,                   
            'topic': get_topic(word),       
            'review_id': index              
        })

df_lens = pd.DataFrame(new_rows)

filename = 'datalens_ready.csv'
df_lens.to_csv(filename, index=False)

print(f"Готово! Файл {filename} создан.")
print(f"В исходной таблице было {len(df)} строк.")
print(f"В таблице для DataLens {len(df_lens)} строк (по количеству слов).")

from google.colab import files
files.download(filename)

Подготовка файла для DataLens...
Готово! Файл datalens_ready.csv создан.
В исходной таблице было 394 строк.
В таблице для DataLens 8208 строк (по количеству слов).


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>